In [1]:
import polars as pl
import os

# Current working directory
current_dir = os.getcwd()

csv_path = os.path.join(current_dir, 'data', 'bank-transaction.csv')

df = pl.read_csv(csv_path, try_parse_dates=True)

In [2]:
month_dict = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}

output_df = (
    df
    .with_columns(
        pl.col('Transaction Code').str.split('-').list.get(0).alias('Bank'),
        pl.col('Transaction Date').dt.month().replace(month_dict).alias('Transaction Month')
    )
    .group_by('Bank', 'Transaction Month')
    .agg(pl.col('Value').sum())
    .with_columns(
        pl.col('Value').rank("dense", descending=True).over('Transaction Month').alias('Bank Rank per Month')
    )
    .with_columns(
        pl.col('Bank Rank per Month').mean().over('Bank').round(2).alias('Avg Rank per Bank'),
        pl.col('Value').mean().over('Bank Rank per Month').round().cast(pl.Int32).alias('Avg Transaction Value per Rank')
    )
)
output_df

Bank,Transaction Month,Value,Bank Rank per Month,Avg Rank per Bank,Avg Transaction Value per Rank
str,str,i64,u32,f64,i32
"""DTB""","""January""",67063,1,1.75,66968
"""DTB""","""October""",58924,2,1.75,48634
"""DSB""","""February""",28622,3,2.33,34621
"""DSB""","""August""",38167,3,2.33,34621
"""DSB""","""May""",45000,2,2.33,48634
…,…,…,…,…,…
"""DS""","""March""",36799,3,1.92,34621
"""DSB""","""January""",45036,3,2.33,34621
"""DS""","""November""",96868,1,1.92,66968
